In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function

import math

import numpy as np
import matplotlib.pyplot as plt
from pytest import approx

from fastimgproto.fixtures.image import (
    add_gaussian2d_to_image,
    gaussian_point_source,
)
from fastimgproto.sourcefind.fit import Gaussian2dParams
from fastimgproto.sourcefind.image import SourceFindImage

from fastimgproto.fixtures.sourcefind import (
    generate_random_source_params,
    check_single_source_extraction_successful,
)

import logging
import datetime

logger = logging.getLogger(__name__)

In [ ]:
%matplotlib inline

In [ ]:
ydim = 64
xdim = 32
image_shape = (ydim, xdim)
seed = 123456

base_x = 18
base_y = 34
n_sources = 500
positive_sources = generate_random_source_params(n_sources=n_sources,
                                                 base_x=base_x,
                                                 base_y=base_y,
                                                 amplitude_range=(5., 42.),
                                                 semiminor_range=(0.7, 2.),
                                                 axis_ratio_range=(1., 5.),
                                                 seed = 123456
                                                )

In [ ]:
positive_sources[0]

In [ ]:
n_islands = 0
island_params = []
fits = []

start = datetime.datetime.now()

for src in positive_sources:
    img = np.zeros(image_shape)
    add_gaussian2d_to_image(src, img)
    detection_thresh = 4.
    sfimg = SourceFindImage(img, detection_n_sigma=detection_thresh,
                            analysis_n_sigma=3.,
                            rms_est=1.,
                            find_negative_sources=True)
    check_single_source_extraction_successful(src, sfimg)
    if sfimg.islands:
        n_islands += 1
        
        assert len(sfimg.islands) == 1
        island_params.append(sfimg.islands[0].params)
        lsq_fit = sfimg.fit_gaussian_2d(sfimg.islands[0], verbose=1)
        fits.append(lsq_fit)
    else:
        islands.append(None)
        fits.append(None)
        
end = datetime.datetime.now()

In [ ]:
duration = (end - start).total_seconds()
n_completed_fits = sum(1 for f in fits if f)  # Count where f is not False
print("{} of {} island-fits completed in {}".format(n_completed_fits, n_islands, duration))

In [ ]:
success = np.zeros_like(positive_sources, dtype=int)
for idx, lsq_fit in enumerate(fits):
    if lsq_fit is None:
        success[idx] = 1 # Trivial no-island case. Count as OK.
    elif (positive_sources[idx].comparable_params ==
            approx(lsq_fit.comparable_params,rel=1e-2, abs=1.0)):
        success[idx] = 1

n_successful = success.sum()
print("{} of {} sources fitted accurately".format(
    n_successful, len(positive_sources)
))

In [ ]:
success.sum()/n_sources

In [ ]:
bad_idx = np.where(success==0)[0]
bad_idx

In [ ]:
import attr

In [ ]:
print(island_params[0])

In [ ]:
num_evaluations = np.array([i.optimize_result.nfev for i in island_params])
np.where(num_evaluations > 10)

In [ ]:
bad_truth = np.array([attr.astuple(s) for s in positive_sources])[bad_idx]
bad_fits = np.array([attr.astuple(s) for s in fits])[bad_idx]

In [ ]:
g2d_parnames = [a.name for a in Gaussian2dParams.__attrs_attrs__]

In [ ]:
import pandas as pd

In [ ]:
diffs = pd.DataFrame(data=(bad_truth - bad_fits),columns=g2d_parnames)
diffs.index=pd.Series(data=bad_idx)
diffs

In [ ]:
for idx in range(len(positive_sources)):
    print(island_params[idx].moments_fit.comparable_params)
    print(positive_sources[idx].comparable_params)
    print(np.degrees(positive_sources[idx].theta), np.degrees(island_params[idx].moments_fit.theta))
    print()

In [ ]:
idx = 178
if idx in range(len(fits)):
    f = fits[idx]
    # print(islands[idx].fit.comparable_params == approx(positive_sources[idx].comparable_params))
    i=island_params[idx]
    print("Peak\n", i.extremum)
    print("Moments\n", i.moments_fit)
    print("Fit\n", f)
    print("Truth\n", positive_sources[idx])
    print()
    print(f.comparable_params)
    print(i.moments_fit.comparable_params)
    print(positive_sources[idx].comparable_params)